In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\DOAN CAO THANH TUYEN\.cache\kagglehub\datasets\grassknoted\asl-alphabet\versions\1


# Import du lieu

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Chuan hoa du lieu

In [14]:
IMG_SIZE = 64
BATCH_SIZE = 32

In [15]:
# cắt tập train : validation + scale ảnh
datagen = ImageDataGenerator(
    rescale=1./255, # chuyen doi gia tri [0, 225] ve [0, 1]
    validation_split=0.2 # chia 
)

In [16]:
train_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # chi dinh tap train
)

val_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # chi dinh tap validation
)

Found 69623 images belonging to 2 classes.
Found 17405 images belonging to 2 classes.


# Khai bao mo hinh - fit

In [17]:
model = Sequential(
    [
        Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2,2)),
        
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        
        Flatten(),
        Dense(512, activation='relu'), # fully connected layer
        Dropout(0.5), # dropout de tranh overfitting
        Dense(29, activation='softmax') # 29 classes trong ASL alphabet
    ]
)

model.summary()

c:\Users\DOAN CAO THANH TUYEN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,467,933 (9.41 MB)

 Trainable params: 2,467,933 (9.41 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# huan luyen
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

Epoch 1/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - accuracy: 0.9997 - loss: 0.0234

# Save model

In [ ]:
model.save('asl_alphabet_model.h5') # .h5: dung de load lai va du doan cho sau nay
print('Saved model')

# Danh gia mo hinh

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {loss:.4f}')

# Du doan

In [ ]:
# load mo hinh
loaded_model = keras.models.load_model('asl_alphabet_model.h5')
print('Loaded model from disk')

In [ ]:
# chuan bi label (chu cai)
class_names = list(train_data.class_indices.keys())
print(class_names)

In [ ]:
# load 1 hiunh de xu ly
img_path = path + '/test.jpg' # duong dan den hinh can du doan
# img_path = input("Enter the path of the image to predict")
img = imgage.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = img.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0)  # them kich thuoc 

In [ ]:
# du doan
predictions = loaded_model.predict(img_array)
predicted_index = np.argmax(predictions)
predicted_label = class_names[predicted_index] # map index -> label

# print % cac lop du doan
print(predictions)
print(f'Predicted label: {predicted_label}')